In [1]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
sns.set_style("darkgrid")

print(os.getcwd())

/home/mgarcia/Documents/Study/DiploDatos/Repositories/G7/DiploDataScienceG7/Mauri/TP3


In [2]:
df = pd.read_csv('./preprocessed_dataset.csv', index_col=0)
new_df = pd.DataFrame()

In [3]:
new_df['total_payment_value'] = df.groupby(["customer_id"])["payment_value"].apply(sum)

In [4]:
new_df['total_buys'] = df.groupby('customer_id')['customer_id'].apply(pd.Series.count)

In [5]:
new_df['prefered_payment_type'] = df.groupby('customer_id')['payment_type'].apply(lambda x:x.mode()[0])

In [6]:
new_df['prefered_payment_installments'] = df.groupby('customer_id')['payment_installments'].apply(lambda x:x.mode()[0])

In [7]:
new_df['mean_review_score'] = df.groupby('customer_id')['review_score'].apply(pd.Series.mean)

In [8]:
new_df['mean_products_photos_qty'] = df.groupby('customer_id')['product_photos_qty'].apply(pd.Series.mean)

In [9]:
new_df['n_different_bought_products'] =  df.groupby('customer_id')['payment_type'].apply(pd.Series.nunique)

In [10]:
new_df['mean_products_volume'] = df.groupby('customer_id')['product_volume'].apply(pd.Series.mean)

In [11]:
new_df['mean_products_weight_g'] = df.groupby('customer_id')['product_weight_g'].apply(pd.Series.mean)

In [12]:
new_df.to_csv('./new_df.csv')

In [13]:
df.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'review_id', 'review_score',
       'review_creation_date', 'review_answer_timestamp',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'seller_zip_code_prefix', 'seller_city',
       'seller_state', 'product_category_name_english', 'Month_order',
       'order_process_time', 'order_delivery_time', 'order_accuracy_time',
       'order_approv

In [14]:
df.product_category_name_english.value_counts(normalize=True)

bed_bath_table               0.103123
health_beauty                0.085893
sports_leisure               0.077338
furniture_decor              0.075316
computers_accessories        0.069945
                               ...   
arts_and_craftmanship        0.000207
la_cuisine                   0.000138
cds_dvds_musicals            0.000120
fashion_childrens_clothes    0.000069
security_and_services        0.000017
Name: product_category_name_english, Length: 71, dtype: float64

In [15]:
# df['total_payment_value'] = df.groupby(["customer_id"])["payment_value"].transform('sum')
# df['total_buys'] = df.groupby('customer_id')['customer_id'].transform('count')
# df['prefered_payment_type'] = df.groupby('customer_id')['payment_type'].transform(lambda x:x.mode()[0])
# df['prefered_payment_installments'] = df.groupby('customer_id')['payment_installments'].transform(lambda x:x.mode()[0])
# df['mean_review_score'] = df.groupby('customer_id')['review_score'].transform('mean')
